In [6]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pystan
import scipy

In [7]:
df = pd.read_excel("/Users/apple/Downloads/userVisits.xlsx")
df.head()

,user_id,total_visits,weeklyVisits,Interval_visits_int
0,1332384,70,"0,0,0,1,2,1,1,0,0,2,1,0,0,2,1,0,0,0,1,1,2,1,0,...","[3, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 1, 4, 2, ..."
1,1332636,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[62, 12, 1, 28]"
2,1334543,4,"1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[6, 0]"
3,1336195,1,"0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",[113]
4,1339702,16,"1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...","[110, 14, 29, 7, 29, 2, 2, 1, 0, 2, 4, 1, 1, 1..."


In [8]:
df = df.loc[df.total_visits > 10]

In [11]:
len(df['total_visits'])

4879

In [12]:
visits = df['weeklyVisits'].tolist()

In [13]:
visits

['0,0,0,1,2,1,1,0,0,2,1,0,0,2,1,0,0,0,1,1,2,1,0,1,0,0,0,0,1,0,0,1,2,1,0,0,1,0,0,0,0,0,0,0,0,2,0,1,1,1,1,1,2,0,1,1,1,0,0,1,0,1,0,0,0,2,1,2,2,2,0,1,0,0,1,1,1,0,1,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,1,1,3,0,0,0,1,0,0,1,1',
 '1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,2,0,1,1,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0',
 '0,1,11,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '0,0,0,0,1,0,0,2,0,1,1,0,0,0,2,0,0,0,3,0,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,2,0,1,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,4,0,0,2,0,1,0,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,2,1,0,0,1,1,0,0,1,0,0,2,1,0,0,0,1',
 '0,3,12,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
visits_int = list(list(int(a) for a in (str(r).split(',')))for r in visits)

In [15]:
len(visits_int)

4879

In [16]:
len(visits_int[10])

113

In [18]:
visitsg_code = """
data {
    int<lower=0> J;// number of users
    int<lower=0> G;// number of groups
    int<lower=0> W;// number of weeks
    int<lower=0> K[G];// number of users in a group
    int y[J,W]; // estimated number of visits next week
    //real<lower=0> sigma[J]; // s.e. of visits estimates
}
parameters {
    //real mu;
    //real<lower=0> tau;
    real<lower=0> eta[J];
    real <lower=0, upper = 5 > etag[G];
    
    //real sigma;
}
transformed parameters {
     real <lower=0> theta[J];
     
     for (j in 1:J)
     theta[j]= eta[j];
}
model {
    //tau ~ normal(0,1);
    
    //for(g in 1:G)
    //etag[g] ~ gamma(1.0,2.0);
    for(g in 1:G){
    for (j in 1:K[g]){
    eta[j] ~ gamma(etag[g],2.0);
    }
    }
    for (j in 1:J){
    for(w in 1:W){
    y[j,w] ~ poisson(theta[j]);
    }
 }
}
"""



visitsg_dat = {'J': len(visits_int),
              'W':113,
              'G':5,
              'K' : [2500,1100,600,400,279],
              'y': visits_int
               }



sm = pystan.StanModel(model_code=visitsg_code)
fit = sm.sampling(data=visitsg_dat, iter=1000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_20442054bb7f5d5494af046a6710e175 NOW.


In [19]:
fit


Inference for Stan model: anon_model_20442054bb7f5d5494af046a6710e175.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

              mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
eta[0]        0.57  1.5e-3   0.07   0.45   0.53   0.57   0.61   0.71   2000    1.0
eta[1]        0.13  7.8e-4   0.04   0.07   0.11   0.13   0.15   0.21   2000    1.0
eta[2]        0.12  7.2e-4   0.03   0.06   0.09   0.11   0.14   0.19   2000    1.0
eta[3]         0.4  1.2e-3   0.06   0.29   0.37    0.4   0.43   0.52   2000    1.0
eta[4]        0.18  8.8e-4   0.04   0.11   0.15   0.18   0.21   0.27   2000    1.0
eta[5]        0.52  1.4e-3   0.06    0.4   0.47   0.51   0.56   0.65   2000    1.0
eta[6]        0.96  2.0e-3   0.09   0.78   0.89   0.95   1.01   1.14   2000    1.0
eta[7]        0.13  7.3e-4   0.03   0.07    0.1   0.12   0.14    0.2   2000    1.0
eta[8]        0.24  1.0e-3   0.05   0.16   0.21   0.24   0.27   0.